In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/income/train.csv')
df.shape

In [ ]:
df.head()

In [ ]:
for col in list(df.columns):
    print(col + ' --> ' + str(df[col].nunique()) + ' , data type '  + str(df[col].dtype))

In [ ]:
cat_columns = [ col for col in list(df.columns) if df[col].dtype =='object']
cat_columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,10))
sns.countplot(data = df, x = cat_columns[0])


In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(data = df, x = cat_columns[1])

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data = df, x = cat_columns[2])

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(data = df, x = cat_columns[3])

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(data = df, x = cat_columns[4])

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(data = df, x = cat_columns[5])

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(data = df, x = cat_columns[6])

In [ ]:
plt.figure(figsize=(15,10))
# selecting top 10 countries 
top_10 = list(df[cat_columns[7]].value_counts().head(10).index)
sns.countplot(data = df.loc[df[cat_columns[7]].isin(top_10)], x = cat_columns[7])

In [ ]:
# Lets drop all Country Except US

df = df[df['native-country']=='United-States']

# drop 'native-country' column from cat_columns list 
df= df.drop('native-country', axis = 1)
cat_columns.remove('native-country')

In [ ]:
#list of categorical 
cat_columns

In [ ]:
num_cols = [ col for col in list(df.columns) if df[col].dtype !='object'  and  col != 'income_>50K']
num_cols


In [ ]:
df_dummy = pd.get_dummies(df[cat_columns])
df_dummy.shape

In [ ]:
print(list(df_dummy.columns))

In [ ]:
#final_dataset

final_df = pd.DataFrame()
final_df = pd.concat([df_dummy , df[num_cols] , df['income_>50K'] ] , axis = 1)
final_df.shape

In [ ]:
list(final_df.columns)

In [ ]:
final_df.info()

In [ ]:
final_df.isna().sum().sum()

In [ ]:
from sklearn.model_selection import train_test_split 
X = final_df.drop('income_>50K', axis =1)
y = final_df['income_>50K']
X_train, X_test, y_train, y_test = train_test_split( X,y , test_size = 0.3, random_state = 0) 
print(X_train.shape)
print(X_test.shape)

In [ ]:
# Training using Decision Tree Classifier 

from sklearn.tree import DecisionTreeClassifier  
classifier1 = DecisionTreeClassifier(criterion='gini')  
classifier1.fit(X_train, y_train) 



In [ ]:

y_pred_1 = classifier1.predict(X_test)  
print(y_pred_1)

In [ ]:
from sklearn.metrics import accuracy_score #importing accuracy_score function from sklearn.metrics package
acc_1 = accuracy_score(y_test,y_pred_1)
print("Accuracy for Gini model {} %".format(acc_1*100))

from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred_1))

In [ ]:
## [[8760 1306]
##  [1141 1981]]


# 8760 -->  people having less $50 k  - rightly predicted  # True Negative  ( target ==0 )
# 1981 -->  people having more $50 k  - rightly predicted  # True Positive  ( target ==1 )

# 1306 -->  people having less $50 k  - model say - they have more than $50K # False Positive 
# 1141 -->  people having more $50 k  - model say - they have less than $50K # False Negative

accuracy  = (8760 + 1981)/(8760+1306+1141+1981)
print(accuracy)

# precision  = True Positive / ( True Positive + False Positive)
precision = 1981/(1981 +1306)
print(precision)

# If I predict 10 people have more than $50K salary - then 60% of that 
#i.e. 6 people will have more than $50K salary

# Recall  = True Positive / (True Positive + False Negative)
recall = 1981/(1981+1141)
print(recall)

# That model will catch 63% of people who have salary more than $50K 


In [ ]:
from sklearn.neighbors import KNeighborsClassifier


classifier2 = KNeighborsClassifier(n_neighbors= 3)  
classifier2.fit(X_train, y_train) 

y_pred_2 = classifier2.predict(X_test)  

acc_2 = accuracy_score(y_test,y_pred_2)
print("Accuracy for KNN model {} %".format(acc_2*100))
print(confusion_matrix(y_test, y_pred_2))

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier3 = LogisticRegression(random_state=0)
classifier3.fit(X_train, y_train) 

y_pred_3 = classifier3.predict(X_test)  

acc_3 = accuracy_score(y_test,y_pred_3)
print("Accuracy for LR model {} %".format(acc_3*100))
print(confusion_matrix(y_test, y_pred_3))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier  

classifier4 = GradientBoostingClassifier()  
classifier4.fit(X_train, y_train) 

y_pred_4 = classifier4.predict(X_test)  

acc_4 = accuracy_score(y_test,y_pred_4)
print("Accuracy for Gradient Boost model {} %".format(acc_4*100))
print(confusion_matrix(y_test, y_pred_4))

In [ ]:
from sklearn.ensemble import RandomForestClassifier  

classifier5 = RandomForestClassifier()  
classifier5.fit(X_train, y_train) 

y_pred_5 = classifier5.predict(X_test)  

acc_5 = accuracy_score(y_test,y_pred_5)
print("Accuracy for Random Forest model {} %".format(acc_5*100))
print(confusion_matrix(y_test, y_pred_5))

In [ ]:
# Lets create a manual ensemble 
from sklearn.metrics import f1_score

ensemble_df = pd.DataFrame()
ensemble_df['Pred1'] = y_pred_1
ensemble_df['Pred2'] = y_pred_2
ensemble_df['Pred3'] = y_pred_3
ensemble_df['Pred4'] = y_pred_4
ensemble_df['Pred5'] = y_pred_5
ensemble_df['Sum'] = ensemble_df.sum(axis = 1)
ensemble_df['Final'] = ensemble_df['Sum'] > 2 
ensemble_df['Final'] = ensemble_df['Final'].astype(int)

print(ensemble_df.head())

acc = accuracy_score(y_test,ensemble_df['Final'])
print("Accuracy for Emsemble model {} %".format(acc*100))
print(confusion_matrix(y_test,ensemble_df['Final']))
print('f1 Score -->' ,f1_score(y_test,ensemble_df['Final']))

In [ ]:
# Lets create a manual ensemble with weighted average

ensemble_df = pd.DataFrame()
ensemble_df['Pred1'] = y_pred_1
ensemble_df['Pred2'] = y_pred_2
ensemble_df['Pred3'] = y_pred_3
ensemble_df['Pred4'] = y_pred_4
ensemble_df['Pred5'] = y_pred_5
# DT 10% , KNN 5%  LR 5%  GB 40% RF 40 % 

ensemble_df['Sum'] = 0.1*ensemble_df['Pred1'] + 0.05*ensemble_df['Pred2'] + \
                     0.05*ensemble_df['Pred3'] + 0.4*ensemble_df['Pred4'] + \
                     0.4*ensemble_df['Pred5']
ensemble_df['Final'] = ensemble_df['Sum'] >= 0.4
ensemble_df['Final'] = ensemble_df['Final'].astype(int)

print(ensemble_df.head())

acc = accuracy_score(y_test,ensemble_df['Final'])
print("Accuracy for Emsemble model {} %".format(acc*100))
print(confusion_matrix(y_test,ensemble_df['Final']))
print('f1 Score -->' ,f1_score(y_test,ensemble_df['Final']))

## Hyper Parameter Tuning of Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV

## base model & parameter grid

In [ ]:
# Base model 
g1 = DecisionTreeClassifier()

param_grid = { 
    'criterion': ['gini', 'entropy'],
    'max_depth': [5,10,15,20,25]
}



## Initiate Grid Search 

In [ ]:
%%time 
gs1 = GridSearchCV(estimator=g1, param_grid=param_grid, cv= 5, verbose = 3)
gs1.fit(X_train, y_train) 

## Best Parameters

In [ ]:
gs1.best_params_

## Best Model 

In [ ]:
best_model_1 = gs1.best_estimator_

y1 = best_model_1.predict(X_test)  

acc = accuracy_score(y_test,y1)
print("Accuracy for Grid Search DT  model {} %".format(acc*100))


print(confusion_matrix(y_test, y1))

## Hyper Parameter Tuning RandomForest 

In [ ]:
# Base model 
g2 = RandomForestClassifier()

param_grid = { 
    'criterion': ['gini', 'entropy'],
    'n_estimators': [50,100,200],
    'max_depth': [5,10,15],
    'max_features': ['auto', 'sqrt', ]
}



In [ ]:
%%time 
gs2 = GridSearchCV(estimator=g2, param_grid=param_grid, cv= 5, verbose = 3)
gs2.fit(X_train, y_train) 

In [ ]:
gs2.best_params_

In [ ]:
best_model_2 = gs2.best_estimator_

y2 = best_model_2.predict(X_test)  

acc = accuracy_score(y_test,y2)
print("Accuracy for Grid Search RF  model {} %".format(acc*100))


print(confusion_matrix(y_test, y2))

## Hyper Parameter Tuning of Gradient Boosting

In [ ]:
# Base model 
g3 = GradientBoostingClassifier()

param_grid = { 
    'n_estimators': [50,100,200],
    'max_depth': [5,10,15],
}



In [ ]:
%%time 
gs3 = GridSearchCV(estimator=g3, param_grid=param_grid, cv= 5, verbose = 3)
gs3.fit(X_train, y_train) 

In [ ]:
gs3.best_params_

In [ ]:
best_model_3 = gs3.best_estimator_

y3 = best_model_3.predict(X_test)  

acc = accuracy_score(y_test,y3)
print("Accuracy for Grid Search Gradient Boosting  model {} %".format(acc*100))


print(confusion_matrix(y_test, y3))

In [ ]:
# # Implementation of SVM 

# from sklearn.svm import SVC
# svc_clf = SVC(C= 1.0 , kernel='poly')

# svc_clf.fit(X_train, y_train)
# svc_pred = svc_clf.predict(X_test)  

# acc_svc = accuracy_score(y_test,svc_pred)
# print("Accuracy for Support Vector Model {} %".format(acc_svc*100))
# print(confusion_matrix(y_test, svc_pred))

In [ ]:
# Scaling  

from sklearn.preprocessing import StandardScaler
sc =StandardScaler()

sc.fit(X_train)
XS_train = sc.transform(X_train)
XS_test = sc.transform(X_test)



In [ ]:
XS_train

In [ ]:
# Implementation of SVM 

from sklearn.svm import SVC
svc_clf = SVC(C= 1.0 , kernel='poly')

svc_clf.fit(XS_train, y_train)
svc_pred = svc_clf.predict(XS_test)  

acc_svc = accuracy_score(y_test,svc_pred)
print("Accuracy for Support Vector Model {} %".format(acc_svc*100))
print(confusion_matrix(y_test, svc_pred))

In [ ]:
from sklearn.svm import SVC
svc_clf = SVC(C= 1.0 , kernel='rbf')

svc_clf.fit(XS_train, y_train)
svc_pred = svc_clf.predict(XS_test)  

acc_svc = accuracy_score(y_test,svc_pred)
print("Accuracy for Support Vector Model {} %".format(acc_svc*100))
print(confusion_matrix(y_test, svc_pred))

In [ ]:
# Data set Info  - check missing values 
# Visulaization  - 
# Transformation  - Dummies 
# Test - Train Split 
# Modeling

# R2 and adj R2  